In [1]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [2]:
MODEL="cardiffnlp/twitter-roberta-base-sentiment"

In [3]:
tokenizer=AutoTokenizer.from_pretrained(MODEL)
model=AutoModelForSequenceClassification.from_pretrained(MODEL)

c:\Users\udayg\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
def result(example):
    text=tokenizer(example,return_tensors='pt')
    output=model(**text)
    scores=output[0][0].detach().numpy()
    score=softmax(scores)
    return score


In [7]:
import json
with open('product_details_with_reviews.json', 'r') as file:
    data = json.load(file)

In [8]:
def remove_surrogates(text):
    return text.encode('utf-16', 'surrogatepass').decode('utf-16')
rating_list = []
title_list = []
price_list = []
polarity_scores=[]
for item in data:
    if 'reviews' in item:
        reviews = item['reviews']
        for review in reviews:
            clean_text = remove_surrogates(review)
        polarity_scores.append(result(clean_text))
    if 'price' in item:
        price = item['price']
        price_list.append(price)

    if 'rating' in item:
        rating = item['rating']
        rating_list.append(rating)  
    if 'title' in item:
        title = item['title']
        title_list.append(title)    


            
print(polarity_scores,price_list,rating_list,title_list)

[array([0.00189828, 0.02640971, 0.97169197], dtype=float32), array([0.00147565, 0.0068026 , 0.9917217 ], dtype=float32), array([0.00894148, 0.05824492, 0.9328136 ], dtype=float32), array([0.04705564, 0.2558488 , 0.6970956 ], dtype=float32), array([0.04705564, 0.2558488 , 0.6970956 ], dtype=float32)] ['1,39,999', '16,999', '6,999', '8,499', '7,699'] ['4.4 out of 5 stars', '3.7 out of 5 stars', '4.0 out of 5 stars', '4.1 out of 5 stars', '4.1 out of 5 stars'] ['Samsung Galaxy S24 Ultra 5G AI Smartphone (Titanium Black, 12GB, 512GB Storage)', 'Redmi Note 13 5G (Arctic White, 6GB RAM, 128GB Storage) | 5G Ready | 120Hz Bezel-Less AMOLED | 7.mm Slimmest Note Ever | 108MP Pro-Grade Camera', 'POCO C65 (Matte Black 4GB RAM 128GB Storage)', 'Redmi 13C (Stardust Black, 6GB RAM, 128GB Storage) | Powered by 4G MediaTek Helio G85 | 90Hz Display | 50MP AI Triple Camera', 'Redmi 13C (Starshine Green, 4GB RAM, 128GB Storage) | Powered by 4G MediaTek Helio G85 | 90Hz Display | 50MP AI Triple Camera']


In [9]:
price_list_int = []
for price in price_list:
    price=int(price.replace(',', ''))
    price_list_int.append(price)
print(price_list_int)
min_price = min(price_list_int)
max_price = max(price_list_int)
final_dict = {}
for i in range(len(title_list)):
        rating = float(rating_list[i].split()[0])

        result = (0.4 * rating -
                  0.2 * (price_list_int[i] - min_price) / (max_price - min_price) +
                  0.4 * (polarity_scores[i][2] - polarity_scores[i][0]))
        
        final_dict[title_list[i]] = result


highest_key = None
highest_value = -100 

for key, value in final_dict.items():
    if value > highest_value:
        highest_value = value
        highest_key = key


print(f"The product with the highest score is: {highest_key} with a score of {highest_value}")



[139999, 16999, 6999, 8499, 7699]
The product with the highest score is: POCO C65 (Matte Black 4GB RAM 128GB Storage) with a score of 1.9695488452911378
